# Solution Planning

## Business Problem

**Qual é o problema de negócios?**
1. Selecionar os clientes mais valiosos para integrar o programa de fidelidade "Loyals".

2. Responder a questões de negócio ao time de marketing.

### Output

**O que vou entregar? / Onde o time de negócio quer ver?**

* 1) Lista em xls / enviar por email. Deve conter clientes que irão aderir ao Loyals (programa de fidelidade). 
   - Formato:
   
| client_id | is_loyal |
|-----------|----------|
|1          |yes       |
|2          |no        |

* 2) Relatório em pdf respondendo as questões de negócio / enviar por email e apresentar ao time de marketing:
    - Quem são as pessoas elegíveis para participar do programa Loyals?
    - Quantos clientes farão parte do grupo?
    - Quais são as principais características desses clientes?
    - Qual a porcentagem de contribuição de faturamento, vinda do Loyals?
    - Qual a expectativa de faturamento desse grupo para os próximos meses?
    - Quais as condições para uma pessoa ser elegível ao Loyals? 
    - Quais as condições para uma pessoa ser removida do Loyals?
    - Qual a garantia que o programa Loyals é melhor que o restante da base?
    - Quais ações do time de marketing pode realizar para aumentar o faturamento?

### Input

**Fontes de dados:**
    Dataset "Ecommerce.csv", contendo as vendas de e-commerce do período de um ano.

**Ferramentas:**
    Python 3.8.12, Jupyter Notebook, Git, Github.

### Process

**Tipo de problema:**
Separação de clientes por grupos.
    
**Principais métodos:**
    Clusterização.

**Perguntas de negócio:**
* 1) Quem são as pessoas elegíveis para participar do programa Loyals?
    - O que é ser elegível? / O que são clientes de maior "valor"? (de acordo com área de negócio) 
        - Faturamento:
             - Alto ticket médio
             - Alto LTV (soma da receita do cliente conosco)
             - Baixa recência (tempo desde a última compra)
             - Alto basket size (qtd produtos comprados por compra)
             - Baixa probabilidade de churn (usaria a saída de um modelo)
             - Alta Previsão LTV (usaria saída de um modelo)
             - Alta propensão de compra (usaria a saída de um modelo)
        - Custo:
             - Baixa taxa de devolução
        - Experiência de compra:
             - Média alta das avaliações

 PS: as features acima serão criadas no feature engeneering. 
 
 
* 2) Quantos clientes farão parte do grupo?
    - Número de clientes
    - % em relação ao total de clients
    
    
* 3) Quais são as principais características desses clientes?
    - Escrever caracterísiticas do cliente:
        - Idade
        - País
        - Salário
        - Localização
     - Escrever os principais comportamentos de compra dos clients ( métricas de negócio )
        - Vide features da clusterização (questão 1)
         
 Para look alike: prospectar clientes parecidos na internet


* 4) Qual a porcentagem de contribuição de faturamento, vinda do Loyals?
    - Calcular o faturamento total da empresa durante o ano.
    - Calcular o faturamento (%) apenas do cluster Loyals.   
   
   
* 5) Qual a expectativa de faturamento desse grupo para os próximos meses?
    - Cálculo do LTV do Loyals (com média móvel, time series, arima..)
    - Séries Temporais ( ARMA, ARIMA, HoltWinter, etc )
    - Análise de Cohort (com tempo, localização, produto..)       

 Deve haver meta de faturamento, consultar negócio.
    
    
* 6) Quais as condições para uma pessoa ser elegível ao Loyals?
    - Definir o período de avaliação (a cada 1 mês, 3 meses..)
    - O "desempenho" do cliente deve estar próximo da média do cluster Loyals.
    
    
* 7) Quais as condições para uma pessoa ser removida do Loyals?
    - O "desempenho" do cliente não está mais próximo da média do cluster Loyals. 
   
   
* 8) Qual a garantia que o programa Loyals é melhor que o restante da base?
    - Teste A/B
    - Teste de hipótese


* 9) Quais ações do time de marketing pode realizar para aumentar o faturamento?
    - Desconto
    - Preferência de compra
    - Frete mais barato
    - Visita a empresa
    - Oferecer personal stylist
    - Recomendar cross sell 
    - Oferecer conteúdo expclusivo

## Solution Benchmarking

### Desk Research

Leitura de artigos sobre customer segmentation na internet, para compreender o que o mercado está fazendo.

Identificar a partir das soluções do mercado, com o time de negócio, o que podemos fazer como MVP.

1. Modelo RFM de segmentação.

# Environment Preparation

## Imports

In [ ]:
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns
from tabulate                 import tabulate
from IPython.core.display     import HTML

#from IPython.display          import Image

## Helper Functions

In [34]:
def jupyter_settings():
    """ Optimize general settings, standardize plot sizes, etc. """
    %matplotlib inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [12, 6]
    plt.rcParams['font.size'] = 20
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.set_option( 'display.expand_frame_repr', False )
    pd.set_option('display.max_columns', 30)
    pd.set_option('display.max_rows', 30)
    sns.set()
jupyter_settings()

# Data Collection

In [35]:
%ls -l ../data/raw/

total 83400
-rw-rw-r--@ 1 home  staff  42697197 Apr 29  2021 Ecommerce.csv


In [36]:
#read data
df_raw = pd.read_csv('../data/raw/Ecommerce.csv', encoding='unicode_escape')

In [37]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,NaN
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN


In [38]:
df_raw = df_raw.drop('Unnamed: 8', axis=1).copy()
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


# Data Description

In [39]:
df1 = df_raw.copy()

## Rename Columns

In [40]:
df1.sample(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
391605,570672,85040A,S/4 PINK FLOWER CANDLES IN BOWL,12,9-Oct-17,1.65,12536.0,France
53380,540832,20726,LUNCH BAG WOODLAND,18,9-Jan-17,4.96,NaN,United Kingdom
170181,551282,21043,APRON MODERN VINTAGE COTTON,4,25-Apr-17,1.95,16919.0,United Kingdom


In [41]:
df1.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [42]:
df1.columns = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country']

## Feature Description 

In [65]:
# Explain feature meanings
tab_meanings = [['Columns', 'Meaning'],
        ['invoice_no', 'unique identifier of each transaction'],
        ['stock_code', 'item code'],
        ['description', 'item name'],
        ['quantity', 'quantity of each item purchased per transaction'],
        ['invoice_date', 'the day the transaction took place'],
        ['unit_price', 'product price per unit'],
        ['customer_id', 'unique customer identifier'],
        ['country', 'customer\'s country of residence']
      ]
print(tabulate(tab_meanings, headers='firstrow', stralign='left', tablefmt='simple'))

Columns       Meaning
------------  -----------------------------------------------
invoice_no    unique identifier of each transaction
stock_code    item code
description   item name
quantity      quantity of each item purchased per transaction
invoice_date  the day the transaction took place
unit_price    product price per unit
customer_id   unique customer identifier
country       customer's country of residence


In [64]:
df1.sample(3)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
126250,547069,22907,PACK OF 20 NAPKINS PANTRY DESIGN,1,18-Mar-17,0.85,16710.0,United Kingdom
294051,562688,21755,LOVE BUILDING BLOCK WORD,2,6-Aug-17,5.95,13869.0,United Kingdom
422121,573038,23355,HOT WATER BOTTLE KEEP CALM,1,25-Oct-17,4.95,17545.0,United Kingdom


## Data Dimensions

In [47]:
print(f'Number of rows: {df1.shape[0]}')
print(f'Number of columns: {df1.shape[1]}')

Number of rows: 541909
Number of columns: 8


In [48]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice_no    541909 non-null  object 
 1   stock_code    541909 non-null  object 
 2   description   540455 non-null  object 
 3   quantity      541909 non-null  int64  
 4   invoice_date  541909 non-null  object 
 5   unit_price    541909 non-null  float64
 6   customer_id   406829 non-null  float64
 7   country       541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## Check NA

In [66]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## Replace NA

In [80]:
#remove rows with NA
df1 = df1.dropna( subset=['description','customer_id'] )

In [100]:
print (f'Removed data: { 1-(df1.shape[0] / df_raw.shape[0]):.2f}%')

Removed data: 0.25%


In [101]:
#check
df1.isna().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
country         0
dtype: int64

## Change Types

In [103]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
customer_id            float64
country                 object
dtype: object

In [52]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


In [78]:
#convert invoice_date to date:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])

In [79]:
# Problem:

#convert invoice_no to int: (need fixing!)
#df1['invoice_no'] = df1['invoice_no'].astype(int)
#ValueError: invalid literal for int() with base 10: 'C536379'

## Descriptive Statistics

## Data Description

# Feature Engeneering

# Variable Filtering

# EDA

# Data Preparation

# Feature Selection

# Model Training

# Cluster Analysis

# Deploy